# Verify Installation

In [1]:
import tokentoss
print(f"tokentoss version: {tokentoss.__version__}")
print(f"Config path: {tokentoss.get_config_path()}")

tokentoss version: 0.1.0
Config path: /Users/nicholasgrundl/Library/Application Support/tokentoss/client_secrets.json


# Check for Existing Config

In [2]:
from pathlib import Path

config_path = tokentoss.get_config_path()
if config_path.exists():
    print(f"Config exists at: {config_path}")
    perms = oct(config_path.stat().st_mode & 0o777)
    print(f"Permissions: {perms}")
else:
    print("No config found — will use ConfigureWidget next")

Config exists at: /Users/nicholasgrundl/Library/Application Support/tokentoss/client_secrets.json
Permissions: 0o600


# ConfigureWidget

In [3]:
from tokentoss import ConfigureWidget

cw = ConfigureWidget()
display(cw)

# Verify Config Was Saved

In [3]:
import json, os

config_path = tokentoss.get_config_path()
assert config_path.exists(), "Config file not created!"

perms = oct(config_path.stat().st_mode & 0o777)
print(f"File: {config_path}")
print(f"Permissions: {perms}")
assert perms == "0o600", f"Expected 0o600, got {perms}"

with open(config_path) as f:
    data = json.load(f)

print(f"Has 'installed' key: {'installed' in data}")
print(f"Client ID ends with: ...{data['installed']['client_id'][-20:]}")
print("Config saved correctly!")

File: /Users/nicholasgrundl/Library/Application Support/tokentoss/client_secrets.json
Permissions: 0o600
Has 'installed' key: True
Client ID ends with: ...oogleusercontent.com
Config saved correctly!


# GoogleAuthWidget — Fresh Auth

In [1]:
import tokentoss
tokentoss.enable_debug() 

In [2]:
from tokentoss import GoogleAuthWidget

widget = GoogleAuthWidget()
display(widget)

[tokentoss.widget DEBUG] Callback server started on port 61894
[tokentoss.widget DEBUG] Started callback server on port 61894


In [3]:
import time                                                          
                                                                       
widget.sign_out()                                                    
widget.prepare_auth()                                                
print(f"Port: {widget._callback_server.port}")                       
print("Complete OAuth in the popup, then come back here...")         
print("Watching server state for 30 seconds:\n")                     
                                                                   
for i in range(30):                                                  
    srv = widget._callback_server                                    
    if srv._server:                                                  
        received = srv._server.callback_received                     
        has_code = srv._server.auth_code is not None                 
    else:                                                            
        received = srv.callback_received                             
        has_code = srv.auth_code is not None                         
    print(f"  [{i}s] received={received}, has_code={has_code}")      
    if received:                                                     
        print(f"  >>> auth_code present: {has_code}")                
        print(f"  >>> error: {srv._server.error if srv._server else srv.error}")                                                         
        break                                                        
    time.sleep(1)                             

[tokentoss.widget DEBUG] prepare_auth: redirect_uri=http://127.0.0.1:61894, state=2QxKQLp9_tiCPmXR5hawaA
Port: 61894
Complete OAuth in the popup, then come back here...
Watching server state for 30 seconds:

  [0s] received=False, has_code=False
  [1s] received=False, has_code=False
  [2s] received=False, has_code=False
  [3s] received=False, has_code=False
  [4s] received=False, has_code=False
  [5s] received=False, has_code=False
[tokentoss.widget DEBUG] do_GET /?state=2QxKQLp9_tiCPmXR5hawaA&code=4%2F0ASc3gC3I7RtILCts9h6cJlevQnZHZ7Y8coKC1JN8zEAXUCpqXYrzaLZPFSCXucVR9wEYvQ&scope=email+profile+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+openid&authuser=0&prompt=consent: code=True, state=True, error=None, is_callback=True
[tokentoss.widget DEBUG] do_GET /favicon.ico: code=False, state=False, error=None, is_callback=False
  [6s] received=True, has_code=True
  >>> auth_code present: True
  >>> error: None


# Verify Authentication State

In [6]:
print(f"is_authenticated: {widget.is_authenticated}")
print(f"user_email: {widget.user_email}")
print(f"status: {widget.status}")

# Check module-level credentials
print(f"\ntokentoss.CREDENTIALS is set: {tokentoss.CREDENTIALS is not None}")
if tokentoss.CREDENTIALS:
    print(f"Credential type: {type(tokentoss.CREDENTIALS).__name__}")
    print(f"Token valid: {tokentoss.CREDENTIALS.valid}")
    print(f"Expired: {tokentoss.CREDENTIALS.expired}")

is_authenticated: True
user_email: nicholasgrundl@gmail.com
status: Signed in as nicholasgrundl@gmail.com

tokentoss.CREDENTIALS is set: True
Credential type: Credentials
Token valid: True
Expired: False


# Inspect AuthManager & Tokens

In [7]:
am = widget.auth_manager

print(f"AuthManager.is_authenticated: {am.is_authenticated}")
print(f"AuthManager.user_email: {am.user_email}")
print(f"Has refresh token: {am.credentials.refresh_token is not None}")

# Check token data in storage
token_data = am.storage.load()
if token_data:
    print(f"\nStored token data:")
    print(f"  user_email: {token_data.user_email}")
    print(f"  scopes: {token_data.scopes}")
    print(f"  expiry: {token_data.expiry}")
    print(f"  is_expired: {token_data.is_expired}")
    print(f"  has id_token: {bool(token_data.id_token)}")
    print(f"  has refresh_token: {bool(token_data.refresh_token)}")

AuthManager.is_authenticated: True
AuthManager.user_email: nicholasgrundl@gmail.com
Has refresh token: True

Stored token data:
  user_email: nicholasgrundl@gmail.com
  scopes: ['https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/userinfo.profile', 'openid']
  expiry: 2026-01-29T21:17:59+00:00
  is_expired: False
  has id_token: True
  has refresh_token: True


# Check Token File on Disk

In [8]:
from tokentoss.storage import FileStorage

fs = FileStorage()
print(f"Token file: {fs.path}")
print(f"Exists: {fs.path.exists()}")
if fs.path.exists():
    perms = oct(fs.path.stat().st_mode & 0o777)
    print(f"Permissions: {perms}")

Token file: /Users/nicholasgrundl/Library/Application Support/tokentoss/tokens.json
Exists: True
Permissions: 0o600


# Sign Out

In [9]:
widget.sign_out()

print(f"is_authenticated: {widget.is_authenticated}")
print(f"user_email: '{widget.user_email}'")
print(f"status: {widget.status}")
print(f"tokentoss.CREDENTIALS: {tokentoss.CREDENTIALS}")

is_authenticated: False
user_email: ''
status: Click to sign in
tokentoss.CREDENTIALS: None


# Re-display Widget After Sign Out

In [10]:
display(widget)

In [13]:
print(f"status: {widget.status}")                   
print(f"error: {widget.error}")                     
print(f"is_authenticated: {widget.is_authenticated}")  

status: Paste the redirect URL below
error: 
is_authenticated: False


In [14]:
widget._check_callback()                            
print(f"status: {widget.status}")                   
print(f"is_authenticated: {widget.is_authenticated}")    

status: Paste the redirect URL below
is_authenticated: False


In [15]:
print(f"Server available: {widget._server_available}")                                          
print(f"Callback server: {widget._callback_server}")                                            
if widget._callback_server:                                                                     
    print(f"Port: {widget._callback_server.port}")                                              
    print(f"Callback received: {widget._callback_server.callback_received}")                    
    print(f"Auth code: {widget._callback_server.auth_code}")                                    
    print(f"Error: {widget._callback_server.error}")

Server available: True
Callback server: <tokentoss.widget.CallbackServer object at 0x11b00f1f0>
Port: 56687
Callback received: False
Auth code: None
Error: None


In [16]:
widget.sign_out()                                                                               
widget._try_start_server()                                                                      
print(f"Server port: {widget._callback_server.port}")                                           
print("Now click 'Sign in with Google' in the widget above") 

Server port: 56820
Now click 'Sign in with Google' in the widget above


In [17]:
print(f"Callback received: {widget._callback_server.callback_received}")                        
print(f"Auth code present: {widget._callback_server.auth_code is not None}") 

Callback received: False
Auth code present: False


In [18]:
print(f"Auth URL redirect: {widget.auth_url}")                                                  
print(f"Server port: {widget._callback_server.port}")                                           
print(f"Server redirect_uri: {widget._callback_server.redirect_uri}")

Auth URL redirect: https://accounts.google.com/o/oauth2/auth?client_id=667572603530-8vo025o6dioqa5849hrc3478t88mg2jg.apps.googleusercontent.com&redirect_uri=http%3A//127.0.0.1%3A56820&response_type=code&scope=openid%20email%20profile&access_type=offline&prompt=consent&code_challenge=TCh3b2i-ea9eDG8S2KZZJ9h1-eJOdxI2yyR4Rq4TmYY&code_challenge_method=S256&state=XeUhlqElXg74ghhpRTvChw
Server port: 56834
Server redirect_uri: http://127.0.0.1:56834


# Verify Re-Auth Worked

In [ ]:
print(f"is_authenticated: {widget.is_authenticated}")
print(f"user_email: {widget.user_email}")
print(f"tokentoss.CREDENTIALS is set: {tokentoss.CREDENTIALS is not None}")